In [1]:
# run stopwatch
from _classes import Stopwatch
stopwatch = Stopwatch()
start = stopwatch.start()

### Load Saved Section if exists

### Load libraries, functions, palette, theme

In [2]:
%run _load.ipynb

In [3]:
economics_data = loadit(
    'economics_data', 'data/00-Datasets/'
)

In [4]:
dir = '003.7.1-003.8.2_SOL-Transferts'

In [5]:
save_df = 'data/00-Datasets/'

In [6]:
save_excel = 'data/00-Excel-data/'

# <font color='#2C8B6D'>I. Население</font>

# Section III. Уровень жизни

## 3. Доходы, расходы и сбережения населения

### 3.7. Объем социальных выплат населению и налогооблагаемых денежных доходов населения в разрезе муниципальных образований. Динамические ряды

*Источник: https://rosstat.gov.ru/folder/13397*  
*Единицы измерения: млрд. руб.*

#### <font color='#2A5899'>Transferts</font>

In [7]:
datasets_list = []

#### <font color='#AF4035'>2010 год</font>

##### Regions OKTMO codes

##### Исправления:
- г. Липецк - неверный ОКТМО

In [8]:
# read file from another directory
path_2010 = \
    ('data/00-Region-codes/')
os.chdir(path_2010)
%run 2010.ipynb
# and go back to current directory
os.chdir(current_path)

##### Data

In [9]:
income_transferts_2010_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2010.xls')

In [10]:
income_transferts_2010_raw = pd.read_excel(income_transferts_2010_link)

In [11]:
income_transferts_2010_raw.head(10)

,Unnamed: 0,Размещено - 16.09.2013г.,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,Объем социальных выплат населению и налогообла...,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,тыс.руб.
3,№,Субъект РФ,Код ОКТМО,Объем социальных выплат и налогооблагаемых ден...,В том числе,NaN
4,NaN,NaN,NaN,NaN,Налогооблагаемые денежные доходы физических л...,Социальные выплаты
5,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,Краснодарский край - всего,3,705518362,556773010,148745352
7,NaN,Муниципальные районы,3600000,289603286,198137507,91465779
8,1,Абинский,3601000,8291272,5429724,2861548
9,2,Апшеронский,3605000,6661006,3784066,2876940


In [12]:
income_transferts_2010_raw.tail(10)

,Unnamed: 0,Размещено - 16.09.2013г.,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
2246,2,Ленинский,99610000,1669910,1208438,461472
2247,3,"""Облученский",99620000,3583876,2695398,888478
2248,4,Октябрьский,99625000,815828,505277,310551
2249,5,Смидовичский,99630000,2237497,1471060,766437
2250,NaN,Городские округа,99700000,13128970,10662169,2466801
2251,1,Местная администрация (исполнительно-распоряди...,99701000,13128970,10662169,2466801
2252,NaN,NaN,NaN,NaN,NaN,NaN
2253,Примечание:,NaN,NaN,NaN,NaN,NaN
2254,NaN,NaN,NaN,NaN,NaN,NaN
2255,Отсутствие информации по другим субъектам Росс...,NaN,NaN,NaN,NaN,NaN


In [13]:
income_transferts_2010 = income_transferts_2010_raw.copy()

##### Rename columns and drop useless

In [14]:
income_transferts_2010_columns = [
    '-', 'index1', 'ОКТМО', 'Суммарные выплаты', 'Доходы', 'Социальные выплаты'
]

In [15]:
income_transferts_2010.columns = income_transferts_2010_columns

In [16]:
# drop useless columns with name '-'
income_transferts_2010 = income_transferts_2010.drop('-', axis=1)

##### Cut rows and drop NaNs

In [17]:
# cut rows
income_transferts_2010 = transform_cut_rows(income_transferts_2010, 6, 4)

In [18]:
# drop rows with NaNs in 'index0' column
income_transferts_2010 = \
    income_transferts_2010[income_transferts_2010['index1'].notna()]

##### Normalize OKTMO

In [19]:
# make OKTMO column values str
income_transferts_2010['ОКТМО'] = income_transferts_2010['ОКТМО'].astype(str)

In [20]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2010['ОКТМО'] = \
    income_transferts_2010['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [21]:
# create index level0 column
income_transferts_2010['index0'] = np.NaN

In [22]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2010 = (income_transferts_2010
                          .assign(index0=income_transferts_2010['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2010['index0'])))

In [23]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2010 = normalize_nenec(income_transferts_2010, 'index0')

In [24]:
# the same correction for 'Тюменская область'
income_transferts_2010 = normalize_khantymanci(income_transferts_2010, 'index0')

In [25]:
# fill '-' values in index0 column
income_transferts_2010 = transform_fill_values_by_previous(
    income_transferts_2010, kind='column', column_name='index0')

##### Level1 for Multiindex

In [26]:
# change region name in 'index1' column to 'Всего'
income_transferts_2010['index1'] = income_transferts_2010.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [27]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2010['index1'] = \
    income_transferts_2010['index1'].apply(lambda x: normalize_municipal_town(x))

##### Count Regions

In [28]:
# count regions
regions_count2010 = len(set(income_transferts_2010['index0'].values))
regions_count2010

66

In [29]:
# create regions list and sort it
regions_list2010 = list(set(income_transferts_2010['index0']))
regions_list2010.sort()

##### Create Multiindex

In [30]:
# columns to multiindex
income_transferts_2010 = transform_make_index_from_columns(
    income_transferts_2010, ['index0', 'index1'])

In [31]:
# drop columns, that became multiindexes
income_transferts_2010 = income_transferts_2010.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [32]:
# round
for column in income_transferts_2010.columns:
    try:
        income_transferts_2010[column] = \
            income_transferts_2010[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2010 Check for Discrepancy</font>

In [33]:
# OKTMO check for NaN
income_transferts_2010[income_transferts_2010['ОКТМО'].isna()].any().any()

False

In [34]:
# all data check for NaN
income_transferts_2010[income_transferts_2010.isna().any(axis=1)].any().any()

False

In [35]:
income_transferts_2010[income_transferts_2010['Доходы'] == 0]

ОКТМО  Суммарные выплаты  Доходы  \
Приморский край   Дальнереченскийрайон  05 610 000                0.0     0.0   
                  Спасскийрайон         05 637 000                0.0     0.0   
                  Шкотовски йрайон      05 657 000                0.0     0.0   
Иркутская область г.Свирск              25 746 000           542373.2     0.0   

                                        Социальные выплаты  
Приморский край   Дальнереченскийрайон                 0.0  
                  Спасскийрайон                        0.0  
                  Шкотовски йрайон                     0.0  
Иркутская область г.Свирск                        542373.2

In [36]:
income_transferts_2010[income_transferts_2010['Социальные выплаты'] == 0]

ОКТМО  Суммарные выплаты  \
Приморский край          Дальнереченскийрайон  05 610 000                0.0   
                         Спасскийрайон         05 637 000                0.0   
                         Шкотовски йрайон      05 657 000                0.0   
Архангельская область    НоваяЗемля            11 712 000          1247155.0   
Пензенская область       Кондольский           56 636 000           268876.7   
Республика Саха (Якутия) Жатай                 98 702 000           731782.1   

                                                  Доходы  Социальные выплаты  
Приморский край          Дальнереченскийрайон        0.0                 0.0  
                         Спасскийрайон               0.0                 0.0  
                         Шкотовски йрайон            0.0                 0.0  
Архангельская область    НоваяЗемля            1247155.0                 0.0  
Пензенская область       Кондольский            268876.7                 0.0  
Республика Саха (Якутия) Жатай                  731782.1                 0.0

In [37]:
income_transferts_2010[income_transferts_2010['Суммарные выплаты'] == 0]

ОКТМО  Суммарные выплаты  Доходы  \
Приморский край Дальнереченскийрайон  05 610 000                0.0     0.0   
                Спасскийрайон         05 637 000                0.0     0.0   
                Шкотовски йрайон      05 657 000                0.0     0.0   

                                      Социальные выплаты  
Приморский край Дальнереченскийрайон                 0.0  
                Спасскийрайон                        0.0  
                Шкотовски йрайон                     0.0

In [38]:
income_transferts_2010[income_transferts_2010['ОКТМО'] == '71']

,,ОКТМО,Суммарные выплаты,Доходы,Социальные выплаты


In [39]:
income_transferts_2010[income_transferts_2010['ОКТМО'] == '11']

,,ОКТМО,Суммарные выплаты,Доходы,Социальные выплаты
Архангельская область,Всего,11,209808384.0,160151998.0,49656386.0
Ненецкий автономный округ (Архангельская область),Всего,11,21400174.0,18694688.0,2705486.0


In [40]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2010_dupl_list = check_oktmo_duplication(
    data=income_transferts_2010, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2010_dupl_list

[]

In [41]:
income_transferts_2010.loc[income_transferts_2010['ОКТМО'].isin(municipal_town_dict['Московская область'])]

ОКТМО  Суммарные выплаты  \
Московская область Муниципальные районы  46 600 000        707497716.5   
                   Городские округа      46 700 000        430931266.0   

                                              Доходы  Социальные выплаты  
Московская область Муниципальные районы  528539493.0         178958223.5  
                   Городские округа      378263156.0          52668110.0

##### <font color='#2A5899'>2010 Final Result</font>

In [42]:
income_transferts_2010.head()

ОКТМО  Суммарные выплаты  \
Краснодарский край Всего                         03        705518362.0   
                   Муниципальные районы  03 600 000        289603286.0   
                   Абинский              03 601 000          8291272.0   
                   Апшеронский           03 605 000          6661006.0   
                   Белоглинский          03 607 000          2875836.0   

                                              Доходы  Социальные выплаты  
Краснодарский край Всего                 556773010.0         148745352.0  
                   Муниципальные районы  198137507.0          91465779.0  
                   Абинский                5429724.0           2861548.0  
                   Апшеронский             3784066.0           2876940.0  
                   Белоглинский            1807010.0           1068826.0

In [43]:
get_data_two_level(
    data=income_transferts_2010,
    level0=['Самарская область', 'Московская область'],
    level1=['Всего', 'Муниципальные районы', 'Городские округа'],
    kind='index')

ОКТМО Суммарные выплаты  \
Самарская область  Всего                         36       580136403.0   
                   Муниципальные районы  36 600 000        63504655.0   
                   Городские округа      36 700 000       516631748.0   
Московская область Всего                         46      1138428982.4   
                   Муниципальные районы  46 600 000       707497716.5   
                   Городские округа      46 700 000       430931266.0   

                                              Доходы Социальные выплаты  
Самарская область  Всего                 476485307.0        103651096.0  
                   Муниципальные районы   41440288.0         22064367.0  
                   Городские округа      435045019.0         81586729.0  
Московская область Всего                 906802649.0        231626333.4  
                   Муниципальные районы  528539493.0        178958223.5  
                   Городские округа      378263156.0         52668110.0

In [44]:
datasets_list.append(income_transferts_2010)

#### <font color='#AF4035'>2011 год</font>

##### Regions OKTMO codes

In [45]:
path_2011 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2011)
%run 2011.ipynb
os.chdir(current_path)

##### Data

In [46]:
income_transferts_2011_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2011.xls')

In [47]:
income_transferts_2011_raw = pd.read_excel(income_transferts_2011_link)

In [48]:
income_transferts_2011_raw.head(10)

,Обновлено - 16.04.2014,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,Объем социальных выплат населению и налогообла...,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,тыс.рублей
2,NaN,NaN,Код ОКТМО,Объем социальных выплат населению и налогообла...,в том числе,NaN
3,NaN,NaN,NaN,NaN,Налогооблагаемые денежные доходы физических ли...,Социальные и другие выплаты
4,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,Алтайский край - всего,01,342913198.515,241197639,101715559.515
6,NaN,Муниципальные районы,01 600 000,92924547.789662,46340970.745412,46583577.04425
7,1,Алейский,01 601 000,1232892.10796,606775.771967,626116.335993
8,2,Алтайский,01 602 000,2142662.396908,1091124.067779,1051538.329129
9,3,Баевский,01 603 000,894644.56671,418110.521243,476534.045467


In [49]:
income_transferts_2011_raw.tail(10)

,Обновлено - 16.04.2014,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
2479,2,Ленинский,99610000,1555226.54166,928803.664559,626422.877101
2480,3,Облученский,99620000,4606157.245012,3423579.864457,1182577.380555
2481,4,Октябрьский,99625000,1017057.174004,606330.803903,410726.3701
2482,5,Смидовичский,99630000,2931305.078043,1927935.28749,1003369.790553
2483,NaN,Городские округа,99700000,15819311.474399,12320798.696102,3498512.778297
2484,1,город Биробиджан,99701000,15819311.474399,12320798.696102,3498512.778297
2485,NaN,NaN,NaN,NaN,NaN,NaN
2486,Примечание:,NaN,NaN,NaN,NaN,NaN
2487,NaN,NaN,NaN,NaN,NaN,NaN
2488,Отсутствие информации по другим субъектам Росс...,NaN,NaN,NaN,NaN,NaN


In [50]:
income_transferts_2011 = income_transferts_2011_raw.copy()

##### Rename columns and drop useless

In [51]:
income_transferts_2011_columns = [
    '-', 'index1', 'ОКТМО', 'Суммарные выплаты', 'Доходы', 'Социальные выплаты'
]

In [52]:
income_transferts_2011.columns = income_transferts_2011_columns

In [53]:
# drop useless columns with name '-'
income_transferts_2011 = income_transferts_2011.drop('-', axis=1)

##### Cut rows and drop NaNs

In [54]:
# cut rows
income_transferts_2011 = transform_cut_rows(income_transferts_2011, 5, 4)

In [55]:
# drop rows with NaNs in 'index0' column
income_transferts_2011 = \
    income_transferts_2011[income_transferts_2011['index1'].notna()]

##### Normalize OKTMO

In [56]:
# make OKTMO column values str
income_transferts_2011['ОКТМО'] = income_transferts_2011['ОКТМО'].astype(str)

In [57]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2011['ОКТМО'] = \
    income_transferts_2011['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [58]:
# create index level0 column
income_transferts_2011['index0'] = np.nan

In [59]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2011 = (income_transferts_2011
                          .assign(index0=income_transferts_2011['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2011['index0'])))

In [60]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2011 = normalize_nenec(income_transferts_2011, 'index0')

In [61]:
# the same correction for 'Тюменская область'
income_transferts_2011 = normalize_khantymanci(income_transferts_2011, 'index0')

In [62]:
# fill '-' values in index0 column
income_transferts_2011 = transform_fill_values_by_previous(
    income_transferts_2011, kind='column', column_name='index0')

##### Level1 for Multiindex

In [63]:
# change region name in 'index1' column to 'Всего'
income_transferts_2011['index1'] = income_transferts_2011.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [64]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2011['index1'] = \
    income_transferts_2011['index1'].apply(lambda x: normalize_municipal_town(x))

##### Count Regions

In [65]:
# count regions
regions_count2011 = len(set(income_transferts_2011['index0'].values))
regions_count2011

74

In [66]:
# create regions list and sort it
regions_list2011 = list(set(income_transferts_2011['index0']))
regions_list2011.sort()

##### Create Multiindex

In [67]:
# columns to multiindex
income_transferts_2011 = transform_make_index_from_columns(
    income_transferts_2011, ['index0', 'index1'])

In [68]:
# drop columns, that became multiindexes
income_transferts_2011 = income_transferts_2011.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [69]:
# round
for column in income_transferts_2011.columns:
    try:
        income_transferts_2011[column] = \
            income_transferts_2011[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2011 Check for Discrepancy</font>

In [70]:
# OKTMO check for NaN
income_transferts_2011[income_transferts_2011['ОКТМО'].isna()].any().any()

False

In [71]:
# all data check for NaN
income_transferts_2011[income_transferts_2011.isna().any(axis=1)].any().any()

False

In [72]:
income_transferts_2011[income_transferts_2011['Доходы'] == 0]

,,ОКТМО,Суммарные выплаты,Доходы,Социальные выплаты
Иркутская область,г.Свирск,25 746 000,720748.1,0.0,720748.1


In [73]:
income_transferts_2011[income_transferts_2011['Социальные выплаты'] == 0]

,,ОКТМО,Суммарные выплаты,Доходы,Социальные выплаты


In [74]:
income_transferts_2011[income_transferts_2011['Суммарные выплаты'] == 0]

,,ОКТМО,Суммарные выплаты,Доходы,Социальные выплаты


In [75]:
income_transferts_2011[income_transferts_2011['ОКТМО'] == '71']

,,ОКТМО,Суммарные выплаты,Доходы,Социальные выплаты


In [76]:
income_transferts_2011[income_transferts_2011['ОКТМО'] == '11']

,,ОКТМО,Суммарные выплаты,Доходы,Социальные выплаты
Архангельская область,Всего,11,240575885.6,166989694.0,73586191.6


In [77]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2011_dupl_list = check_oktmo_duplication(
    data=income_transferts_2011, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2011_dupl_list

[]

In [78]:
(income_transferts_2011
 .loc[income_transferts_2011['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО  Суммарные выплаты  \
Алтайский край Муниципальные районы  01 600 000         92924547.8   
               Городские округа      01 700 000        249988650.7   

                                          Доходы  Социальные выплаты  
Алтайский край Муниципальные районы   46340970.7          46583577.0  
               Городские округа      194856668.3          55131982.5

##### <font color='#2A5899'>2011 Final Result</font>

In [79]:
income_transferts_2011.head()

ОКТМО  Суммарные выплаты  \
Алтайский край Всего                         01        342913198.5   
               Муниципальные районы  01 600 000         92924547.8   
               Алейский              01 601 000          1232892.1   
               Алтайский             01 602 000          2142662.4   
               Баевский              01 603 000           894644.6   

                                          Доходы  Социальные выплаты  
Алтайский край Всего                 241197639.0         101715559.5  
               Муниципальные районы   46340970.7          46583577.0  
               Алейский                 606775.8            626116.3  
               Алтайский               1091124.1           1051538.3  
               Баевский                 418110.5            476534.0

In [80]:
get_data_two_level(
    data=income_transferts_2011,
    level0=['Алтайский край', 'Московская область'],
    level1=['Всего', 'Муниципальные районы', 'Городские округа'],
    kind='index')

ОКТМО Суммарные выплаты  \
Алтайский край     Всего                         01       342913198.5   
                   Муниципальные районы  01 600 000        92924547.8   
                   Городские округа      01 700 000       249988650.7   
Московская область Всего                         46      1412884618.6   
                   Муниципальные районы  46 600 000       850362146.3   
                   Городские округа      46 700 000       562522472.3   

                                               Доходы Социальные выплаты  
Алтайский край     Всего                  241197639.0        101715559.5  
                   Муниципальные районы    46340970.7         46583577.0  
                   Городские округа       194856668.3         55131982.5  
Московская область Всего                 1104331594.0        308553024.6  
                   Муниципальные районы   661139782.0        189222364.3  
                   Городские округа       443191812.0        119330660.3

In [81]:
datasets_list.append(income_transferts_2011)

#### <font color='#AF4035'>2012 год</font>

##### Regions OKTMO codes

In [82]:
path_2012 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2012)
%run 2012.ipynb
os.chdir(current_path)

##### Data

In [83]:
income_transferts_2012_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2012.xls')

In [84]:
income_transferts_2012_raw = pd.read_excel(income_transferts_2012_link)

In [85]:
income_transferts_2012_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,50624588.047421,50214690.56262,100839278.610041,93639.957739
6,1,Алейский,01 601 000,656390.280557,665344.849647,1321735.130204,84688.609611
7,2,Алтайский,01 602 000,1576860.348383,1150522.195613,2727382.543995,105630.617506
8,3,Баевский,01 603 000,506281.097062,521166.373618,1027447.47068,99050.175521
9,4,Бийский,01 604 000,1148975.868953,1545738.537512,2694714.406465,80367.265329


In [86]:
income_transferts_2012_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2634,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2635,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2636,NaN,Муниципальные районы,99600000,9832107.841731,4156179.546254,13988287.387985,143221.363872
2637,1,Биробиджанский,99605000,1219048.141402,519834.052221,1738882.193624,145330.730767
2638,2,Ленинский,99610000,1315161.697156,697282.492625,2012444.189782,100541.776068
2639,3,Облученский,99620000,4850010.001627,1301999.321632,6152009.323259,221678.05287
2640,4,Октябрьский,99625000,660611.905604,481984.245302,1142596.150906,105913.621701
2641,5,Смидовичский,99630000,1787276.095941,1155079.434474,2942355.530414,108382.036629
2642,NaN,Городские округа,99700000,15234962.658269,3786680.582426,19021643.240695,250703.718592
2643,1,г. Биробиджан,99701000,15234962.658269,3786680.582426,19021643.240695,250703.718592


In [87]:
income_transferts_2012 = income_transferts_2012_raw.copy()

##### Rename columns and drop useless

In [88]:
income_transferts_2012_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [89]:
income_transferts_2012.columns = income_transferts_2012_columns

In [90]:
# drop useless columns with name '-'
income_transferts_2012 = income_transferts_2012.drop('-', axis=1)

##### Cut rows and drop NaNs

In [91]:
# cut rows
income_transferts_2012 = transform_cut_rows(income_transferts_2012, 4, 0)

In [92]:
# drop rows with NaNs in 'index0' column
income_transferts_2012 = \
    income_transferts_2012[income_transferts_2012['index1'].notna()]

##### Normalize OKTMO

In [93]:
# make OKTMO column values str
income_transferts_2012['ОКТМО'] = income_transferts_2012['ОКТМО'].astype(str)

In [94]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2012['ОКТМО'] = \
    income_transferts_2012['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [95]:
# create index level0 column
income_transferts_2012['index0'] = np.NaN

In [96]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2012 = (income_transferts_2012
                          .assign(index0=income_transferts_2012['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2012['index0'])))

In [97]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2012 = normalize_nenec(income_transferts_2012, 'index0')

In [98]:
# the same correction for 'Тюменская область'
income_transferts_2012 = normalize_khantymanci(income_transferts_2012, 'index0')

In [99]:
# fill '-' values in index0 column
income_transferts_2012 = transform_fill_values_by_previous(
    income_transferts_2012, kind='column', column_name='index0')

##### Level1 for Multiindex

In [100]:
# change region name in 'index1' column to 'Всего'
income_transferts_2012['index1'] = income_transferts_2012.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [101]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2012['index1'] = \
    income_transferts_2012['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [102]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [103]:
# create regions list
regions_list2012 = list(set(income_transferts_2012['index0']))
regions_list2012.sort()

In [104]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2012 = calculate_sum_transferts(
    data=income_transferts_2012,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [105]:
# count regions
regions_count2012 = len(set(income_transferts_2012['index0'].values))
regions_count2012

82

##### Create Multiindex

In [106]:
# columns to multiindex
income_transferts_2012 = transform_make_index_from_columns(
    income_transferts_2012, ['index0', 'index1'])

In [107]:
# drop columns, that became multiindexes
income_transferts_2012 = income_transferts_2012.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [108]:
# round
for column in income_transferts_2012.columns:
    try:
        income_transferts_2012[column] = \
            income_transferts_2012[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2012 Check for Discrepancy</font>

In [109]:
# OKTMO check for NaN
income_transferts_2012[income_transferts_2012['ОКТМО'].isna()].any().any()

False

In [110]:
# all data check for NaN
income_transferts_2012[income_transferts_2012.isna().any(axis=1)].any().any()

True

In [111]:
income_transferts_2012[income_transferts_2012['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Иркутская область,г.Свирск,25 746 000,0.0,199437.7,199437.7,15078.1


In [112]:
income_transferts_2012[income_transferts_2012['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [113]:
income_transferts_2012[income_transferts_2012['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [114]:
income_transferts_2012[income_transferts_2012['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [115]:
income_transferts_2012[income_transferts_2012['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,253938964.1,65538442.9,319477407.0,413988.2
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,731329664.0,95417787.8,826747451.8,1040050.6
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,303687048.2,34685584.9,338372633.2,1364070.9


In [116]:
income_transferts_2012[income_transferts_2012['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,184116980.0,82476202.3,266593182.3,433861.5
Ненецкий автономный округ (Архангельская область),Всего,11,46435432.0,7865292.3,54300724.3,2538953.5


In [117]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2012_dupl_list = check_oktmo_duplication(
    data=income_transferts_2012, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2012_dupl_list

['Ненецкий автономный округ (Архангельская область)']

In [118]:
(income_transferts_2012
 .loc[income_transferts_2012['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   50624588.0   
               Городские округа      01 700 000  142422952.0   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          50214690.6        100839278.6   
               Городские округа              58505288.4        200928240.4   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы            93640.0  
               Городские округа               151517.4

##### <font color='#2A5899'>2012 Final Result</font>

In [119]:
income_transferts_2012.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  193047540.0   
               Муниципальные районы  01 600 000   50624588.0   
               Алейский              01 601 000     656390.3   
               Алтайский             01 602 000    1576860.3   
               Баевский              01 603 000     506281.1   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        108719979.0        301767519.0   
               Муниципальные районы          50214690.6        100839278.6   
               Алейский                        665344.8          1321735.1   
               Алтайский                      1150522.2          2727382.5   
               Баевский                        521166.4          1027447.5   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          245157.3  
               Муниципальные районы            93640.0  
               Алейский                        84688.6  
               Алтайский                      105630.6  
               Баевский                        99050.2

In [120]:
get_data_two_level(
    data=income_transferts_2012,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,1178230608.0,352392692.8,1530623300.8,441876.5


In [121]:
datasets_list.append(income_transferts_2012)

#### <font color='#AF4035'>2013 год</font>

##### Regions OKTMO codes

In [122]:
path_2013 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2013)
%run 2013.ipynb
os.chdir(current_path)

##### Data

In [123]:
income_transferts_2013_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2013.xls')

In [124]:
income_transferts_2013_raw = pd.read_excel(income_transferts_2013_link)

In [125]:
income_transferts_2013_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,54597561.905445,57995716.76011,112593278.665555,105816.474208
6,1,Алейский,01 601 000,746209.968536,770101.451626,1516311.420163,99456.34397
7,2,Алтайский,01 602 000,1561849.131594,1309969.785414,2871818.917008,110833.966926
8,3,Баевский,01 603 000,476578.519131,598545.885224,1075124.404355,106680.333832
9,4,Бийский,01 604 000,1314510.906119,1859907.002281,3174417.9084,95993.767831


In [126]:
income_transferts_2013_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2626,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2627,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2628,NaN,Муниципальные районы,99600000,11263093.139224,4832458.511891,16095551.651116,167039.079797
2629,1,Биробиджанский,99605000,1697873.413127,621598.539945,2319471.953072,192104.683872
2630,2,Ленинский,99610000,1918050.167754,819001.792116,2737051.959869,138830.938872
2631,3,Облученский,99620000,4799494.181507,1528100.942312,6327595.123819,232307.626251
2632,4,Октябрьский,99625000,938660.08494,542722.789388,1481382.874328,139214.629671
2633,5,Смидовичский,99630000,1909015.291898,1321034.448131,3230049.740028,121020.971901
2634,NaN,Городские округа,99700000,15162345.160776,4423486.759219,19585831.919994,260567.702419
2635,1,г.Биробиджан,99701000,15162345.160776,4423486.759219,19585831.919994,260567.702419


In [127]:
income_transferts_2013 = income_transferts_2013_raw.copy()

##### Rename columns and drop useless

In [128]:
income_transferts_2013_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [129]:
income_transferts_2013.columns = income_transferts_2013_columns

In [130]:
# drop useless columns with name '-'
income_transferts_2013 = income_transferts_2013.drop('-', axis=1)

##### Cut rows and drop NaNs

In [131]:
# cut rows
income_transferts_2013 = transform_cut_rows(income_transferts_2013, 4, 0)

In [132]:
# drop rows with NaNs in 'index0' column
income_transferts_2013 = \
    income_transferts_2013[income_transferts_2013['index1'].notna()]

##### Normalize OKTMO

In [133]:
# make OKTMO column values str
income_transferts_2013['ОКТМО'] = income_transferts_2013['ОКТМО'].astype(str)

In [134]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2013['ОКТМО'] = \
    income_transferts_2013['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [135]:
# create index level0 column
income_transferts_2013['index0'] = np.NaN

In [136]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2013 = (income_transferts_2013
                          .assign(index0=income_transferts_2013['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2013['index0'])))

In [137]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2013 = normalize_nenec(income_transferts_2013, 'index0')

In [138]:
# the same correction for 'Тюменская область'
income_transferts_2013 = normalize_khantymanci(income_transferts_2013, 'index0')

In [139]:
# fill '-' values in index0 column
income_transferts_2013 = transform_fill_values_by_previous(
    income_transferts_2013, kind='column', column_name='index0')

##### Level1 for Multiindex

In [140]:
# change region name in 'index1' column to 'Всего'
income_transferts_2013['index1'] = income_transferts_2013.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [141]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2013['index1'] = \
    income_transferts_2013['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [142]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [143]:
# create regions list
regions_list2012 = list(set(income_transferts_2013['index0']))
regions_list2012.sort()

In [144]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2013 = calculate_sum_transferts(
    data=income_transferts_2013,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [145]:
# count regions
regions_count2012 = len(set(income_transferts_2013['index0'].values))
regions_count2012

82

##### Create Multiindex

In [146]:
# columns to multiindex
income_transferts_2013 = transform_make_index_from_columns(
    income_transferts_2013, ['index0', 'index1'])

In [147]:
# drop columns, that became multiindexes
income_transferts_2013 = income_transferts_2013.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [148]:
# round
for column in income_transferts_2013.columns:
    try:
        income_transferts_2013[column] = \
            income_transferts_2013[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2013 Check for Discrepancy</font>

In [149]:
# OKTMO check for NaN
income_transferts_2013[income_transferts_2013['ОКТМО'].isna()].any().any()

False

In [150]:
# all data check for NaN
income_transferts_2013[income_transferts_2013.isna().any(axis=1)].any().any()

True

In [151]:
income_transferts_2013[income_transferts_2013['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [152]:
income_transferts_2013[income_transferts_2013['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [153]:
income_transferts_2013[income_transferts_2013['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [154]:
income_transferts_2013[income_transferts_2013['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [155]:
income_transferts_2013[income_transferts_2013['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,275021927.7,73305712.8,348327640.5,441621.6
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,732913677.0,110784034.4,843697711.4,1069051.6
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,349768170.1,41329294.3,391097464.4,1564536.8


In [156]:
income_transferts_2013[income_transferts_2013['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,205356512.0,92571963.1,297928475.1,493355.3
Ненецкий автономный округ (Архангельская область),Всего,11,55326046.0,8649153.0,63975199.0,2912213.0


In [157]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2013_dupl_list = check_oktmo_duplication(
    data=income_transferts_2013, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2013_dupl_list

['Ненецкий автономный округ (Архангельская область)']

In [158]:
(income_transferts_2013
 .loc[income_transferts_2013['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   54597561.9   
               Городские округа      01 700 000  167307437.5   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          57995716.8        112593278.7   
               Городские округа              69059039.7        236366477.2   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           105816.5  
               Городские округа               177632.2

##### <font color='#2A5899'>2013 Final Result</font>

In [159]:
income_transferts_2013.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  221904999.4   
               Муниципальные районы  01 600 000   54597561.9   
               Алейский              01 601 000     746210.0   
               Алтайский             01 602 000    1561849.1   
               Баевский              01 603 000     476578.5   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        127054756.5        348959755.9   
               Муниципальные районы          57995716.8        112593278.7   
               Алейский                        770101.5          1516311.4   
               Алтайский                      1309969.8          2871818.9   
               Баевский                        598545.9          1075124.4   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          283448.7  
               Муниципальные районы           105816.5  
               Алейский                        99456.3  
               Алтайский                      110834.0  
               Баевский                       106680.3

In [160]:
get_data_two_level(
    data=income_transferts_2013,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,1361849160.9,408993192.1,1770842353.0,503093.9


In [161]:
datasets_list.append(income_transferts_2013)

#### <font color='#AF4035'>2014 год</font>

##### Regions OKTMO codes

In [162]:
path_2014 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2014)
%run 2014.ipynb
os.chdir(current_path)

##### Data

In [163]:
income_transferts_2014_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2014.xls')

In [164]:
income_transferts_2014_raw = pd.read_excel(income_transferts_2014_link)

In [165]:
income_transferts_2014_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,56152781,61319439.770861,117472220.770861,111533.613519
6,1,Алейский,01 601 000,751737,807012.558656,1558749.558656,103647.154642
7,2,Алтайский,01 602 000,2249835,1398079.147707,3647914.147707,139981.356397
8,3,Баевский,01 603 000,459865,618096.024901,1077961.024901,109749.646192
9,4,Бийский,01 604 000,1370409,1993439.810531,3363848.810531,102628.331163


In [166]:
income_transferts_2014_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2627,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2628,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2629,NaN,Муниципальные районы,99600000,10694149.5,4995713.631691,15689863.131691,165874.077659
2630,1,Биробиджанский,99605000,1439763,622877.616913,2062640.616913,172375.114233
2631,2,Ленинский,99610000,1736981.6,830660.809395,2567642.409395,133307.845356
2632,3,Облученский,99620000,4923678.3,1626804.733701,6550483.033701,244585.282417
2633,4,Октябрьский,99625000,772833.8,550173.029273,1323006.829273,126980.212043
2634,5,Смидовичский,99630000,1820892.8,1365197.442409,3186090.242409,121787.784963
2635,NaN,Городские округа,99700000,15125481,4735712.922719,19861193.922719,265580.79165
2636,1,г.Биробиджан,99701000,15125481,4735712.922719,19861193.922719,265580.79165


In [167]:
income_transferts_2014 = income_transferts_2014_raw.copy()

##### Rename columns and drop useless

In [168]:
income_transferts_2014_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [169]:
income_transferts_2014.columns = income_transferts_2014_columns

In [170]:
# drop useless columns with name '-'
income_transferts_2014 = income_transferts_2014.drop('-', axis=1)

##### Cut rows and drop NaNs

In [171]:
# cut rows
income_transferts_2014 = transform_cut_rows(income_transferts_2014, 4, 0)

In [172]:
# drop rows with NaNs in 'index0' column
income_transferts_2014 = \
    income_transferts_2014[income_transferts_2014['index1'].notna()]

##### Normalize OKTMO

In [173]:
# make OKTMO column values str
income_transferts_2014['ОКТМО'] = income_transferts_2014['ОКТМО'].astype(str)

In [174]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2014['ОКТМО'] = \
    income_transferts_2014['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [175]:
# create index level0 column
income_transferts_2014['index0'] = np.NaN

In [176]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2014 = (income_transferts_2014
                          .assign(index0=income_transferts_2014['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2014['index0'])))

In [177]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2014 = normalize_nenec(income_transferts_2014, 'index0')

In [178]:
# the same correction for 'Тюменская область'
income_transferts_2014 = normalize_khantymanci(income_transferts_2014, 'index0')

In [179]:
# fill '-' values in index0 column
income_transferts_2014 = transform_fill_values_by_previous(
    income_transferts_2014, kind='column', column_name='index0')

##### Level1 for Multiindex

In [180]:
# change region name in 'index1' column to 'Всего'
income_transferts_2014['index1'] = income_transferts_2014.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [181]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2014['index1'] = \
    income_transferts_2014['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [182]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [183]:
# create regions list
regions_list2012 = list(set(income_transferts_2014['index0']))
regions_list2012.sort()

In [184]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2014 = calculate_sum_transferts(
    data=income_transferts_2014,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [185]:
# count regions
regions_count2012 = len(set(income_transferts_2014['index0'].values))
regions_count2012

82

##### Create Multiindex

In [186]:
# columns to multiindex
income_transferts_2014 = transform_make_index_from_columns(
    income_transferts_2014, ['index0', 'index1'])

In [187]:
# drop columns, that became multiindexes
income_transferts_2014 = income_transferts_2014.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [188]:
# round
for column in income_transferts_2014.columns:
    try:
        income_transferts_2014[column] = \
            income_transferts_2014[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2014 Check for Discrepancy</font>

In [189]:
# OKTMO check for NaN
income_transferts_2014[income_transferts_2014['ОКТМО'].isna()].any().any()

False

In [190]:
# all data check for NaN
income_transferts_2014[income_transferts_2014.isna().any(axis=1)].any().any()

True

In [191]:
income_transferts_2014[income_transferts_2014['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [192]:
income_transferts_2014[income_transferts_2014['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [193]:
income_transferts_2014[income_transferts_2014['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [194]:
income_transferts_2014[income_transferts_2014['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [195]:
income_transferts_2014[income_transferts_2014['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,273241349.0,87404144.8,360645493.8,456135.6
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,646014842.4,125084303.1,771099145.5,1033500.4
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,352327224.1,45884045.8,398211269.9,1617387.6


In [196]:
income_transferts_2014[income_transferts_2014['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,230911090.0,96654055.8,327565145.8,545281.8
Ненецкий автономный округ (Архангельская область),Всего,11,62794340.0,8519788.9,71314128.9,3203344.6


In [197]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2014_dupl_list = check_oktmo_duplication(
    data=income_transferts_2014, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2014_dupl_list

['Ненецкий автономный округ (Архангельская область)']

In [198]:
(income_transferts_2014
 .loc[income_transferts_2014['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   56152781.0   
               Городские округа      01 700 000  168854960.0   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          61319439.8        117472220.8   
               Городские округа              73217670.2        242072630.2   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           111533.6  
               Городские округа               181398.5

##### <font color='#2A5899'>2014 Final Result</font>

In [199]:
income_transferts_2014.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  225007741.0   
               Муниципальные районы  01 600 000   56152781.0   
               Алейский              01 601 000     751737.0   
               Алтайский             01 602 000    2249835.0   
               Баевский              01 603 000     459865.0   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        134537110.0        359544851.0   
               Муниципальные районы          61319439.8        117472220.8   
               Алейский                        807012.6          1558749.6   
               Алтайский                      1398079.1          3647914.1   
               Баевский                        618096.0          1077961.0   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          292932.1  
               Муниципальные районы           111533.6  
               Алейский                       103647.2  
               Алтайский                      139981.4  
               Баевский                       109749.6

In [200]:
get_data_two_level(
    data=income_transferts_2014,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,1483266710.7,451593916.2,1934860626.9,541140.0


In [201]:
datasets_list.append(income_transferts_2014)

#### <font color='#AF4035'>2015 год</font>

##### Regions OKTMO codes

In [202]:
path_2015 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2015)
%run 2015.ipynb
os.chdir(current_path)

##### Data

In [203]:
income_transferts_2015_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2015.xls')

In [204]:
income_transferts_2015_raw = pd.read_excel(income_transferts_2015_link)

In [205]:
income_transferts_2015_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,65678390.200549,70320487.235812,135998877.436361,125150.805606
6,1,Алейский,01 601 000,934172.675937,871714.118465,1805886.794402,121731.499454
7,2,Алтайский,01 602 000,1197788.022308,1556525.711051,2754313.733359,105408.103075
8,3,Баевский,01 603 000,621466.733172,683757.069359,1305223.802531,135890.036703
9,4,Бийский,01 604 000,1538278.541479,2112479.545443,3650758.086922,111767.024459


In [206]:
income_transferts_2015_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2652,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2653,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2654,NaN,Муниципальные районы,99600000,10679283.6,5759075.796397,16438359.396397,177566.101326
2655,1,Биробиджанский,99605000,1377187.1,838266.687595,2215453.787595,189031.893139
2656,2,Ленинский,99610000,1832316.5,924636.02968,2756952.52968,147218.055731
2657,3,Облученский,99620000,4530068.7,1810012.108117,6340080.808117,239909.214369
2658,4,Октябрьский,99625000,924954.7,620776.735547,1545731.435547,151869.860046
2659,5,Смидовичский,99630000,2014756.6,1565384.235459,3580140.835459,140265.665078
2660,NaN,Городские округа,99700000,15127271.8,5144344.049003,20271615.849003,271490.007085
2661,1,г. Биробиджан,99701000,15127271.8,5144344.049003,20271615.849003,271490.007085


In [207]:
income_transferts_2015 = income_transferts_2015_raw.copy()

##### Rename columns and drop useless

In [208]:
income_transferts_2015_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [209]:
income_transferts_2015.columns = income_transferts_2015_columns

In [210]:
# drop useless columns with name '-'
income_transferts_2015 = income_transferts_2015.drop('-', axis=1)

##### Cut rows and drop NaNs

In [211]:
# cut rows
income_transferts_2015 = transform_cut_rows(income_transferts_2015, 4, 0)

In [212]:
# drop rows with NaNs in 'index0' column
income_transferts_2015 = \
    income_transferts_2015[income_transferts_2015['index1'].notna()]

##### Normalize OKTMO

In [213]:
# make OKTMO column values str
income_transferts_2015['ОКТМО'] = income_transferts_2015['ОКТМО'].astype(str)

In [214]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2015['ОКТМО'] = \
    income_transferts_2015['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [215]:
# create index level0 column
income_transferts_2015['index0'] = np.NaN

In [216]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2015 = (income_transferts_2015
                          .assign(index0=income_transferts_2015['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2015['index0'])))

In [217]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2015 = normalize_nenec(income_transferts_2015, 'index0')

In [218]:
# the same correction for 'Тюменская область'
income_transferts_2015 = normalize_khantymanci(income_transferts_2015, 'index0')

In [219]:
# fill '-' values in index0 column
income_transferts_2015 = transform_fill_values_by_previous(
    income_transferts_2015, kind='column', column_name='index0')

##### Level1 for Multiindex

In [220]:
# change region name in 'index1' column to 'Всего'
income_transferts_2015['index1'] = income_transferts_2015.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [221]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2015['index1'] = \
    income_transferts_2015['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [222]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [223]:
# create regions list
regions_list2012 = list(set(income_transferts_2015['index0']))
regions_list2012.sort()

In [224]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2015 = calculate_sum_transferts(
    data=income_transferts_2015,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [225]:
# count regions
regions_count2012 = len(set(income_transferts_2015['index0'].values))
regions_count2012

83

##### Create Multiindex

In [226]:
# columns to multiindex
income_transferts_2015 = transform_make_index_from_columns(
    income_transferts_2015, ['index0', 'index1'])

In [227]:
# drop columns, that became multiindexes
income_transferts_2015 = income_transferts_2015.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [228]:
# round
for column in income_transferts_2015.columns:
    try:
        income_transferts_2015[column] = \
            income_transferts_2015[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2015 Check for Discrepancy</font>

In [229]:
# OKTMO check for NaN
income_transferts_2015[income_transferts_2015['ОКТМО'].isna()].any().any()

False

In [230]:
# all data check for NaN
income_transferts_2015[income_transferts_2015.isna().any(axis=1)].any().any()

True

In [231]:
income_transferts_2015[income_transferts_2015['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [232]:
income_transferts_2015[income_transferts_2015['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [233]:
income_transferts_2015[income_transferts_2015['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [234]:
income_transferts_2015[income_transferts_2015['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [235]:
income_transferts_2015[income_transferts_2015['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,274668222.4,99837484.9,374505707.3,469860.5
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,668197077.6,153447030.9,821644108.5,1118045.1
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,369587579.6,55319778.6,424907358.2,1753025.6


In [236]:
income_transferts_2015[income_transferts_2015['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,247978617.0,110044136.8,358022753.8,598399.3
Ненецкий автономный округ (Архангельская область),Всего,11,63835520.0,12223102.4,76058622.4,4393722.2


In [237]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2015_dupl_list = check_oktmo_duplication(
    data=income_transferts_2015, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2015_dupl_list

['Ненецкий автономный округ (Архангельская область)']

In [238]:
(income_transferts_2015
 .loc[income_transferts_2015['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   65678390.2   
               Городские округа      01 700 000  169286143.8   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          70320487.2        135998877.4   
               Городские округа              83426554.0        252712697.8   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           125150.8  
               Городские округа               195278.7

##### <font color='#2A5899'>2015 Final Result</font>

In [239]:
income_transferts_2015.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  234964534.0   
               Муниципальные районы  01 600 000   65678390.2   
               Алейский              01 601 000     934172.7   
               Алтайский             01 602 000    1197788.0   
               Баевский              01 603 000     621466.7   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        153747041.2        388711575.2   
               Муниципальные районы          70320487.2        135998877.4   
               Алейский                        871714.1          1805886.8   
               Алтайский                      1556525.7          2754313.7   
               Баевский                        683757.1          1305223.8   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          320429.5  
               Муниципальные районы           125150.8  
               Алейский                       121731.5  
               Алтайский                      105408.1  
               Баевский                       135890.0

In [240]:
get_data_two_level(
    data=income_transferts_2015,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,1573541583.1,569337289.6,2142878872.6,590808.6


In [241]:
datasets_list.append(income_transferts_2015)

#### <font color='#AF4035'>2016 год</font>

##### Regions OKTMO codes

In [242]:
path_2016 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2016)
%run 2016.ipynb
os.chdir(current_path)

##### Исправления:
- Осетия-Алания: Городские округа: '90 701 000' -> '90 700 000'
- Магаданская область: '44 000 000' -> '44'

##### Data

In [243]:
income_transferts_2016_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2016.xls')

In [244]:
income_transferts_2016_raw = pd.read_excel(income_transferts_2016_link)

In [245]:
income_transferts_2016_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,73099701.749086,76278727.009065,149378428.758151,138526.279984
6,1,Алейский,01 601 000,1190669.884214,933376.281442,2124046.165656,145373.086418
7,2,Алтайский,01 602 000,1758606.298665,1690557.705343,3449164.004008,132375.038533
8,3,Баевский,01 603 000,626780.512525,754054.859022,1380835.371548,146336.940605
9,4,Бийский,01 604 000,1686703.310148,2242627.126207,3929330.436356,120995.548464


In [246]:
income_transferts_2016_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2653,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2654,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2655,NaN,Муниципальные районы,99600000,11346326.57,6106105.787262,17452432.357262,192118.539412
2656,1,Биробиджанский,99605000,1405277.07,819129.682514,2224406.752514,193645.577828
2657,2,Ленинский,99610000,2227391.51,1012860.08579,3240251.59579,177373.089325
2658,3,Облученский,99620000,4390040.5,1932859.497599,6322899.997599,241885.998378
2659,4,Октябрьский,99625000,1092235.73,682886.24521,1775121.97521,177388.025903
2660,5,Смидовичский,99630000,2231381.76,1658370.276149,3889752.036149,155964.395996
2661,NaN,Городские округа,99700000,14767906.72,5876774.392738,20644681.112738,277754.801253
2662,1,г. Биробиджан,99701000,14767906.72,5876774.392738,20644681.112738,277754.801253


In [247]:
income_transferts_2016 = income_transferts_2016_raw.copy()

##### Rename columns and drop useless

In [248]:
income_transferts_2016_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [249]:
income_transferts_2016.columns = income_transferts_2016_columns

In [250]:
# drop useless columns with name '-'
income_transferts_2016 = income_transferts_2016.drop('-', axis=1)

##### Cut rows and drop NaNs

In [251]:
# cut rows
income_transferts_2016 = transform_cut_rows(income_transferts_2016, 4, 0)

In [252]:
# drop rows with NaNs in 'index0' column
income_transferts_2016 = \
    income_transferts_2016[income_transferts_2016['index1'].notna()]

##### Normalize OKTMO

In [253]:
# make OKTMO column values str
income_transferts_2016['ОКТМО'] = income_transferts_2016['ОКТМО'].astype(str)

In [254]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2016['ОКТМО'] = \
    income_transferts_2016['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [255]:
# create index level0 column
income_transferts_2016['index0'] = np.NaN

In [256]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2016 = (income_transferts_2016
                          .assign(index0=income_transferts_2016['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2016['index0'])))

In [257]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2016 = normalize_nenec(income_transferts_2016, 'index0')

In [258]:
# the same correction for 'Тюменская область'
income_transferts_2016 = normalize_khantymanci(income_transferts_2016, 'index0')

In [259]:
# fill '-' values in index0 column
income_transferts_2016 = transform_fill_values_by_previous(
    income_transferts_2016, kind='column', column_name='index0')

##### Level1 for Multiindex

In [260]:
# change region name in 'index1' column to 'Всего'
income_transferts_2016['index1'] = income_transferts_2016.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [261]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2016['index1'] = \
    income_transferts_2016['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [262]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [263]:
# create regions list
regions_list2012 = list(set(income_transferts_2016['index0']))
regions_list2012.sort()

In [264]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2016 = calculate_sum_transferts(
    data=income_transferts_2016,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [265]:
# count regions
regions_count2012 = len(set(income_transferts_2016['index0'].values))
regions_count2012

83

##### Create Multiindex

In [266]:
# columns to multiindex
income_transferts_2016 = transform_make_index_from_columns(
    income_transferts_2016, ['index0', 'index1'])

In [267]:
# drop columns, that became multiindexes
income_transferts_2016 = income_transferts_2016.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [268]:
# round
for column in income_transferts_2016.columns:
    try:
        income_transferts_2016[column] = \
            income_transferts_2016[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2016 Check for Discrepancy</font>

In [269]:
# OKTMO check for NaN
income_transferts_2016[income_transferts_2016['ОКТМО'].isna()].any().any()

False

In [270]:
# all data check for NaN
income_transferts_2016[income_transferts_2016.isna().any(axis=1)].any().any()

True

In [271]:
income_transferts_2016[income_transferts_2016['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [272]:
income_transferts_2016[income_transferts_2016['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [273]:
income_transferts_2016[income_transferts_2016['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [274]:
income_transferts_2016[income_transferts_2016['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [275]:
income_transferts_2016[income_transferts_2016['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,294774745.0,103552744.5,398327489.6,493370.8
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,716732014.2,150878663.8,867610678.0,1151022.1
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,401211839.0,51898186.0,453110024.9,1890530.5


In [276]:
income_transferts_2016[income_transferts_2016['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,250969951.4,111891133.0,362861084.3,617488.2
Ненецкий автономный округ (Архангельская область),Всего,11,61562798.0,11794261.5,73357059.6,3409809.3


In [277]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2016_dupl_list = check_oktmo_duplication(
    data=income_transferts_2016, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2016_dupl_list

['Ненецкий автономный округ (Архангельская область)']

In [278]:
income_transferts_2016.loc[income_transferts_2016['ОКТМО'].isin(municipal_town_dict['Алтайский край'])]

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   73099701.7   
               Городские округа      01 700 000  178095155.8   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          76278727.0        149378428.8   
               Городские округа              88801735.6        266896891.5   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           138526.3  
               Городские округа               206434.8

##### <font color='#2A5899'>2016 Final Result</font>

In [279]:
income_transferts_2016.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  251194857.6   
               Муниципальные районы  01 600 000   73099701.7   
               Алейский              01 601 000    1190669.9   
               Алтайский             01 602 000    1758606.3   
               Баевский              01 603 000     626780.5   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        165080462.6        416275320.2   
               Муниципальные районы          76278727.0        149378428.8   
               Алейский                        933376.3          2124046.2   
               Алтайский                      1690557.7          3449164.0   
               Баевский                        754054.9          1380835.4   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          344961.1  
               Муниципальные районы           138526.3  
               Алейский                       145373.1  
               Алтайский                      132375.0  
               Баевский                       146336.9

In [280]:
get_data_two_level(
    data=income_transferts_2016,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,1713242333.5,600921192.2,2314163526.2,628822.0


In [281]:
income_transferts_2016.loc['Ненецкий автономный округ (Архангельская область)']

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,11,61562798.0,11794261.5,73357059.6,3409809.3
Муниципальные районы,11 810 000,17134716.0,1982165.3,19116881.3,990871.4
Заполярны йрайон,11 810 000,17134716.0,1982165.3,19116881.3,990871.4
Городские округа,11 850 000,13646683.0,3914965.4,17561648.4,714033.3
г.Нарьян-Мар,11 850 000,13646683.0,3914965.4,17561648.4,714033.3


In [282]:
datasets_list.append(income_transferts_2016)

#### <font color='#AF4035'>2017 год</font>

##### Regions OKTMO codes

In [283]:
path_2017 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2017)
%run 2017.ipynb
os.chdir(current_path)

##### Исправления:
- Осетия-Алания: Городские округа: '90 701 000' -> '90 700 000'
- г. Магадан: '44 700 000' -> '44 701 000'

##### Data

In [284]:
income_transferts_2017_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2017.xls')

In [285]:
income_transferts_2017_raw = pd.read_excel(income_transferts_2017_link)

In [286]:
income_transferts_2017_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,72304088.765149,76060472.993595,148364561.758745,138784.282722
6,1,Алейский,01 601 000,1225368.205378,912457.927912,2137826.13329,147844.130933
7,2,Алтайский,01 602 000,2248113.45868,1666497.296198,3914610.754878,150828.80307
8,3,Баевский,01 603 000,570225.684025,749286.655353,1319512.339378,141883.047245
9,4,Бийский,01 604 000,1737433.319328,2423657.937774,4161091.257102,129713.870666


In [287]:
income_transferts_2017_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2653,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2654,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2655,NaN,Муниципальные районы,99600000,11393785,6076410.02461,17470195.02461,195729.130764
2656,1,Биробиджанский,99605000,1143200,806014.149622,1949214.149622,172466.302391
2657,2,Ленинский,99610000,1605069,971295.571494,2576364.571494,143626.077126
2658,3,Облученский,99620000,5188191,1973850.729384,7162041.729384,278040.363732
2659,4,Октябрьский,99625000,1174452,654799.313254,1829251.313254,186145.447568
2660,5,Смидовичский,99630000,2282873,1670450.260855,3953323.260855,161815.859394
2661,NaN,Городские округа,99700000,15130946,5865429.683424,20996375.683424,284276.468452
2662,1,г. Биробиджан,99701000,15130946,5865429.683424,20996375.683424,284276.468452


In [288]:
income_transferts_2017 = income_transferts_2017_raw.copy()

##### Rename columns and drop useless

In [289]:
income_transferts_2017_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [290]:
income_transferts_2017.columns = income_transferts_2017_columns

In [291]:
# drop useless columns with name '-'
income_transferts_2017 = income_transferts_2017.drop('-', axis=1)

##### Cut rows and drop NaNs

In [292]:
# cut rows
income_transferts_2017 = transform_cut_rows(income_transferts_2017, 4, 0)

In [293]:
# drop rows with NaNs in 'index0' column
income_transferts_2017 = \
    income_transferts_2017[income_transferts_2017['index1'].notna()]

##### Normalize OKTMO

In [294]:
# make OKTMO column values str
income_transferts_2017['ОКТМО'] = income_transferts_2017['ОКТМО'].astype(str)

In [295]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2017['ОКТМО'] = \
    income_transferts_2017['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [296]:
# create index level0 column
income_transferts_2017['index0'] = np.NaN

In [297]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2017 = (income_transferts_2017
                          .assign(index0=income_transferts_2017['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2017['index0'])))

In [298]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2017 = normalize_nenec(income_transferts_2017, 'index0')

In [299]:
# the same correction for 'Тюменская область'
income_transferts_2017 = normalize_khantymanci(income_transferts_2017, 'index0')

In [300]:
# fill '-' values in index0 column
income_transferts_2017 = transform_fill_values_by_previous(
    income_transferts_2017, kind='column', column_name='index0')

##### Level1 for Multiindex

In [301]:
# change region name in 'index1' column to 'Всего'
income_transferts_2017['index1'] = income_transferts_2017.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [302]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2017['index1'] = \
    income_transferts_2017['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [303]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [304]:
# create regions list
regions_list2012 = list(set(income_transferts_2017['index0']))
regions_list2012.sort()

In [305]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2017 = calculate_sum_transferts(
    data=income_transferts_2017,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [306]:
# count regions
regions_count2012 = len(set(income_transferts_2017['index0'].values))
regions_count2012

83

##### Create Multiindex

In [307]:
# columns to multiindex
income_transferts_2017 = transform_make_index_from_columns(
    income_transferts_2017, ['index0', 'index1'])

In [308]:
# drop columns, that became multiindexes
income_transferts_2017 = income_transferts_2017.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [309]:
# round
for column in income_transferts_2017.columns:
    try:
        income_transferts_2017[column] = \
            income_transferts_2017[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2016 Check for Discrepancy</font>

In [310]:
# OKTMO check for NaN
income_transferts_2017[income_transferts_2017['ОКТМО'].isna()].any().any()

False

In [311]:
# all data check for NaN
income_transferts_2017[income_transferts_2017.isna().any(axis=1)].any().any()

True

In [312]:
income_transferts_2017[income_transferts_2017['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [313]:
income_transferts_2017[income_transferts_2017['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [314]:
income_transferts_2017[income_transferts_2017['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [315]:
income_transferts_2017[income_transferts_2017['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [316]:
income_transferts_2017[income_transferts_2017['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,348148209.0,102876100.4,451024309.4,540182.7
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,744842711.5,159809528.6,904652240.1,1195748.7
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,434032932.5,58250172.4,492283104.9,2065803.5


In [317]:
income_transferts_2017[income_transferts_2017['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,256421310.1,119016584.2,375437894.2,646180.6
Ненецкий автономный округ (Архангельская область),Всего,11,29711549.6,7234095.4,36945645.0,1712736.6


In [318]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2016_dupl_list = check_oktmo_duplication(
    data=income_transferts_2017, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2016_dupl_list

['Ростовская область']

In [319]:
(income_transferts_2017
 .loc[income_transferts_2017['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   72304088.8   
               Городские округа      01 700 000  179762250.2   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          76060473.0        148364561.8   
               Городские округа              90611540.4        270373790.6   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           138784.3  
               Городские округа               209779.1

##### <font color='#2A5899'>2016 Final Result</font>

In [320]:
income_transferts_2017.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  252066339.0   
               Муниципальные районы  01 600 000   72304088.8   
               Алейский              01 601 000    1225368.2   
               Алтайский             01 602 000    2248113.5   
               Баевский              01 603 000     570225.7   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        166672013.4        418738352.4   
               Муниципальные районы          76060473.0        148364561.8   
               Алейский                        912457.9          2137826.1   
               Алтайский                      1666497.3          3914610.8   
               Баевский                        749286.7          1319512.3   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          348563.4  
               Муниципальные районы           138784.3  
               Алейский                       147844.1  
               Алтайский                      150828.8  
               Баевский                       141883.0

In [321]:
get_data_two_level(
    data=income_transferts_2017,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,1604609424.2,589393407.0,2194002831.2,600261.9


In [322]:
income_transferts_2017.loc['Ненецкий автономный округ (Архангельская область)']

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,11,29711549.6,7234095.4,36945645.0,1712736.6
Муниципальные районы,11 800 000,16243530.6,2732327.7,18975858.3,985656.5
Заполярны йрайон,11 810 000,16243530.6,2732327.7,18975858.3,985656.5
Городские округа,11 900 000,13468019.0,4501767.7,17969786.7,727080.2
г.Нарьян-Мар,11 850 000,13468019.0,4501767.7,17969786.7,727080.2


In [323]:
datasets_list.append(income_transferts_2017)

#### <font color='#AF4035'>2018 год</font>

##### Regions OKTMO codes

In [324]:
path_2018 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2018)
%run 2018.ipynb
os.chdir(current_path)

##### Исправления:
- Магаданская область: '44 000 000' -> '44'

##### Data

In [325]:
income_transferts_2018_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2018.xls')

In [326]:
income_transferts_2018_raw = pd.read_excel(income_transferts_2018_link)

In [327]:
income_transferts_2018_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,79730737.796009,77754431.413884,157485169.209893,149068.606612
6,1,Алейский,01 601 000,1140899.02366,930983.623306,2071882.646966,145548.4824
7,2,Алтайский,01 602 000,1549957.652244,1925262.073505,3475219.725749,134318.390822
8,3,Баевский,01 603 000,711078.914745,774420.925473,1485499.840218,162740.999147
9,4,Бийский,01 604 000,1807290.303886,2468633.376222,4275923.680108,135562.858414


In [328]:
income_transferts_2018_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2657,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2658,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2659,NaN,Муниципальные районы,99600000,12568118.212837,6114883.226784,18683001.439621,213305.492072
2660,1,Биробиджанский,99605000,1091896.525415,784342.556083,1876239.081499,167851.05399
2661,2,Ленинский,99610000,1980212.587744,983554.236092,2963766.823836,168261.997493
2662,3,Облученский,99620000,5597990.402031,1977347.47262,7575337.874651,300275.006923
2663,4,Октябрьский,99625000,1376482.000563,631494.824726,2007976.82529,208512.650601
2664,5,Смидовичский,99630000,2521536.697083,1738144.137262,4259680.834345,177946.396288
2665,NaN,Городские округа,99700000,16820048.787163,6161316.270227,22981365.05739,313200.025313
2666,1,г. Биробиджан,99701000,16820048.787163,6161316.270227,22981365.05739,313200.025313


In [329]:
income_transferts_2018 = income_transferts_2018_raw.copy()

##### Rename columns and drop useless

In [330]:
income_transferts_2018_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [331]:
income_transferts_2018.columns = income_transferts_2018_columns

In [332]:
# drop useless columns with name '-'
income_transferts_2018 = income_transferts_2018.drop('-', axis=1)

##### Cut rows and drop NaNs

In [333]:
# cut rows
income_transferts_2018 = transform_cut_rows(income_transferts_2018, 4, 0)

In [334]:
# drop rows with NaNs in 'index0' column
income_transferts_2018 = \
    income_transferts_2018[income_transferts_2018['index1'].notna()]

##### Normalize OKTMO

In [335]:
# make OKTMO column values str
income_transferts_2018['ОКТМО'] = income_transferts_2018['ОКТМО'].astype(str)

In [336]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2018['ОКТМО'] = \
    income_transferts_2018['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [337]:
# create index level0 column
income_transferts_2018['index0'] = np.NaN

In [338]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2018 = (income_transferts_2018
                          .assign(index0=income_transferts_2018['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2018['index0'])))

In [339]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2018 = normalize_nenec(income_transferts_2018, 'index0')

In [340]:
# the same correction for 'Тюменская область'
income_transferts_2018 = normalize_khantymanci(income_transferts_2018, 'index0')

In [341]:
# fill '-' values in index0 column
income_transferts_2018 = transform_fill_values_by_previous(
    income_transferts_2018, kind='column', column_name='index0')

##### Level1 for Multiindex

In [342]:
# change region name in 'index1' column to 'Всего'
income_transferts_2018['index1'] = income_transferts_2018.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [343]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2018['index1'] = \
    income_transferts_2018['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [344]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [345]:
# create regions list
regions_list2012 = list(set(income_transferts_2018['index0']))
regions_list2012.sort()

In [346]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2018 = calculate_sum_transferts(
    data=income_transferts_2018,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [347]:
# count regions
regions_count2012 = len(set(income_transferts_2018['index0'].values))
regions_count2012

83

##### Create Multiindex

In [348]:
# columns to multiindex
income_transferts_2018 = transform_make_index_from_columns(
    income_transferts_2018, ['index0', 'index1'])

In [349]:
# drop columns, that became multiindexes
income_transferts_2018 = income_transferts_2018.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [350]:
# round
for column in income_transferts_2018.columns:
    try:
        income_transferts_2018[column] = \
            income_transferts_2018[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2016 Check for Discrepancy</font>

In [351]:
# OKTMO check for NaN
income_transferts_2018[income_transferts_2018['ОКТМО'].isna()].any().any()

False

In [352]:
# all data check for NaN
income_transferts_2018[income_transferts_2018.isna().any(axis=1)].any().any()

True

In [353]:
income_transferts_2018[income_transferts_2018['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [354]:
income_transferts_2018[income_transferts_2018['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [355]:
income_transferts_2018[income_transferts_2018['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [356]:
income_transferts_2018[income_transferts_2018['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [357]:
income_transferts_2018[income_transferts_2018['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,362880697.0,106125982.9,469006679.9,557018.6
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,795579169.3,170849001.9,966428171.2,1309106.1
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,459337412.7,59484873.1,518822285.8,2172085.7


In [358]:
income_transferts_2018[income_transferts_2018['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,273905569.0,117547544.3,391453113.3,693277.8
Ненецкий автономный округ (Архангельская область),Всего,11,34295455.4,7074370.1,41369825.4,1939209.3


In [359]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2016_dupl_list = check_oktmo_duplication(
    data=income_transferts_2018, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2016_dupl_list

['Ростовская область']

In [360]:
(income_transferts_2018
 .loc[income_transferts_2018['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   79730737.8   
               Городские округа      01 700 000  217344940.5   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          77754431.4        157485169.2   
               Городские округа              91732606.3        309077546.8   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           149068.6  
               Городские округа               240529.9

##### <font color='#2A5899'>2016 Final Result</font>

In [361]:
income_transferts_2018.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  297075678.3   
               Муниципальные районы  01 600 000   79730737.8   
               Алейский              01 601 000    1140899.0   
               Алтайский             01 602 000    1549957.7   
               Баевский              01 603 000     711078.9   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        169487037.7        466562716.0   
               Муниципальные районы          77754431.4        157485169.2   
               Алейский                        930983.6          2071882.6   
               Алтайский                      1925262.1          3475219.7   
               Баевский                        774420.9          1485499.8   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          389598.5  
               Муниципальные районы           149068.6  
               Алейский                       145548.5  
               Алтайский                      134318.4  
               Баевский                       162741.0

In [362]:
get_data_two_level(
    data=income_transferts_2018,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,2062209378.2,628545704.8,2690755083.0,681857.2


In [363]:
income_transferts_2018.loc['Ненецкий автономный округ (Архангельская область)']

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,11,34295455.4,7074370.1,41369825.4,1939209.3
Муниципальные районы,11 800 000,20023658.4,2567080.7,22590739.0,1182018.6
Заполярны йрайон,11 810 000,20023658.4,2567080.7,22590739.0,1182018.6
Городские округа,11 900 000,14271797.0,4507289.4,18779086.4,757190.7
г.Нарьян-Мар,11 850 000,14271797.0,4507289.4,18779086.4,757190.7


In [364]:
datasets_list.append(income_transferts_2018)

#### <font color='#AF4035'>2019 год</font>

##### Regions OKTMO codes

In [365]:
path_2019 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2019)
%run 2019.ipynb
os.chdir(current_path)

##### Исправления:
- Магаданская область: '44 000 000' -> '44'

##### Data

In [366]:
income_transferts_2019_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2019.xls')

In [367]:
income_transferts_2019_raw = pd.read_excel(income_transferts_2019_link)

In [368]:
income_transferts_2019_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,87555253.578974,81599427.089409,169154680.668383,162270.983286
6,1,Алейский,1601000,1456673.759432,991512.965766,2448186.725198,176496.772057
7,2,Алтайский,1602000,1596343.617729,1819335.140655,3415678.758384,131884.580809
8,3,Баевский,1603000,848164.137064,828642.90007,1676807.037133,187814.408281
9,4,Бийский,1604000,2194324.400856,2381892.414764,4576216.81562,147273.092898


In [369]:
income_transferts_2019_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2636,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2637,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2638,NaN,Муниципальные районы,99600000,13521287,6526101.95676,20047388.95676,231432.632866
2639,1,Биробиджанский,99605000,1170077.7,892138.331706,2062216.031706,186659.669778
2640,2,Ленинский,99610000,1823731.4,1057618.213701,2881349.613701,164338.653608
2641,3,Облученский,99620000,6376265.5,2093230.836635,8469496.336635,340852.235055
2642,4,Октябрьский,99625000,1252798.9,701527.180169,1954326.080169,205913.610807
2643,5,Смидовичский,99630000,2898413.5,1781587.394549,4680000.894549,197443.39934
2644,NaN,Городские округа,99700000,17402772,6586773.02324,23989545.02324,330954.184577
2645,1,г. Биробиджан,99701000,17402772,6586773.02324,23989545.02324,330954.184577


In [370]:
income_transferts_2019 = income_transferts_2019_raw.copy()

##### Rename columns and drop useless

In [371]:
income_transferts_2019_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [372]:
income_transferts_2019.columns = income_transferts_2019_columns

In [373]:
# drop useless columns with name '-'
income_transferts_2019 = income_transferts_2019.drop('-', axis=1)

##### Cut rows and drop NaNs

In [374]:
# cut rows
income_transferts_2019 = transform_cut_rows(income_transferts_2019, 4, 0)

In [375]:
# drop rows with NaNs in 'index0' column
income_transferts_2019 = \
    income_transferts_2019[income_transferts_2019['index1'].notna()]

##### Normalize OKTMO

In [376]:
# make OKTMO column values str
income_transferts_2019['ОКТМО'] = income_transferts_2019['ОКТМО'].astype(str)

In [377]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2019['ОКТМО'] = \
    income_transferts_2019['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [378]:
# create index level0 column
income_transferts_2019['index0'] = np.NaN

In [379]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2019 = (income_transferts_2019
                          .assign(index0=income_transferts_2019['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2019['index0'])))

In [380]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2019 = normalize_nenec(income_transferts_2019, 'index0')

In [381]:
# the same correction for 'Тюменская область'
income_transferts_2019 = normalize_khantymanci(income_transferts_2019, 'index0')

In [382]:
# fill '-' values in index0 column
income_transferts_2019 = transform_fill_values_by_previous(
    income_transferts_2019, kind='column', column_name='index0')

##### Level1 for Multiindex

In [383]:
# change region name in 'index1' column to 'Всего'
income_transferts_2019['index1'] = income_transferts_2019.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [384]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2019['index1'] = \
    income_transferts_2019['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [385]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [386]:
# create regions list
regions_list2012 = list(set(income_transferts_2019['index0']))
regions_list2012.sort()

In [387]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2019 = calculate_sum_transferts(
    data=income_transferts_2019,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [388]:
# count regions
regions_count2012 = len(set(income_transferts_2019['index0'].values))
regions_count2012

83

##### Create Multiindex

In [389]:
# columns to multiindex
income_transferts_2019 = transform_make_index_from_columns(
    income_transferts_2019, ['index0', 'index1'])

In [390]:
# drop columns, that became multiindexes
income_transferts_2019 = income_transferts_2019.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [391]:
# round
for column in income_transferts_2019.columns:
    try:
        income_transferts_2019[column] = \
            income_transferts_2019[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2016 Check for Discrepancy</font>

In [392]:
# OKTMO check for NaN
income_transferts_2019[income_transferts_2019['ОКТМО'].isna()].any().any()

False

In [393]:
# all data check for NaN
income_transferts_2019[income_transferts_2019.isna().any(axis=1)].any().any()

True

In [394]:
income_transferts_2019[income_transferts_2019['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [395]:
income_transferts_2019[income_transferts_2019['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [396]:
income_transferts_2019[income_transferts_2019['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [397]:
income_transferts_2019[income_transferts_2019['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [398]:
income_transferts_2019[income_transferts_2019['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,399692830.4,115110090.3,5.148029e+08,600785.8
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,858195228.5,180597145.4,1.038792e+09,1400606.8
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,471454094.1,65782413.5,5.372365e+08,2229393.3


In [399]:
income_transferts_2019[income_transferts_2019['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,296658826.2,123841698.2,420500524.3,730442.7
Ненецкий автономный округ (Архангельская область),Всего,11,35221374.0,7497958.2,42719332.2,2008437.5


In [400]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2016_dupl_list = check_oktmo_duplication(
    data=income_transferts_2019, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2016_dupl_list

['Ростовская область']

In [401]:
(income_transferts_2019
 .loc[income_transferts_2019['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   87555253.6   
               Городские округа      01 700 000  234061837.6   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          81599427.1        169154680.7   
               Городские округа              95459379.0        329521216.6   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           162271.0  
               Городские округа               256924.2

##### <font color='#2A5899'>2016 Final Result</font>

In [402]:
income_transferts_2019.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  321617091.2   
               Муниципальные районы  01 600 000   87555253.6   
               Алейский              01 601 000    1456673.8   
               Алтайский             01 602 000    1596343.6   
               Баевский              01 603 000     848164.1   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        177058806.1        498675897.3   
               Муниципальные районы          81599427.1        169154680.7   
               Алейский                        991513.0          2448186.7   
               Алтайский                      1819335.1          3415678.8   
               Баевский                        828642.9          1676807.0   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          419195.2  
               Муниципальные районы           162271.0  
               Алейский                       176496.8  
               Алтайский                      131884.6  
               Баевский                       187814.4

In [403]:
get_data_two_level(
    data=income_transferts_2019,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,2450614543.0,653789986.9,3104404529.9,829543.6


In [404]:
income_transferts_2019.loc['Ненецкий автономный округ (Архангельская область)']

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,11,35221374.0,7497958.2,42719332.2,2008437.5
Муниципальные районы,11 800 000,20891011.0,2707329.4,23598340.4,1243261.2
Заполярны йрайон,11 810 000,20891011.0,2707329.4,23598340.4,1243261.2
Городские округа,11 900 000,14330363.0,4790628.8,19120991.8,765176.3
г.Нарьян-Мар,11 850 000,14330363.0,4790628.8,19120991.8,765176.3


In [405]:
datasets_list.append(income_transferts_2019)

#### <font color='#AF4035'>2020 год</font>

##### Regions OKTMO codes

In [406]:
path_2020 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2020)
%run 2020.ipynb
os.chdir(current_path)

##### Исправления:
- Магаданская область: '44 000 000' -> '44'

##### Data

In [407]:
income_transferts_2020_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2020.xlsx')

In [408]:
income_transferts_2020_raw = pd.read_excel(income_transferts_2020_link)

In [409]:
income_transferts_2020_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,98592778.521697,92751510.905266,191344289.426963,186104.33575
6,1,Алейский,1601000,1591328.890211,1135974.872515,2727303.762726,201962.660154
7,2,Алтайский,1602000,1769736.827467,2092842.416862,3862579.24433,148298.366134
8,3,Баевский,1603000,804125.320942,916818.893085,1720944.214027,197991.741144
9,4,Бийский,1604000,2305381.830704,2868774.9765,5174156.807204,169538.871103


In [410]:
income_transferts_2020_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2634,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2635,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2636,NaN,Муниципальные районы,99600000,14560368.2,7717067.937107,22277436.137107,258247.196248
2637,1,Биробиджанский,99605000,1257015.3,1028987.943429,2286003.243429,209034.678441
2638,2,Ленинский,99610000,1892686.1,1332471.351879,3225157.451879,182470.011422
2639,3,Облученский,99620000,7018416.6,2398803.291252,9417219.891252,382378.589055
2640,4,Октябрьский,99625000,1325533.5,823927.997465,2149461.497465,228666.116752
2641,5,Смидовичский,99630000,3066716.7,2132877.353082,5199594.053082,220088.637167
2642,NaN,Городские округа,99700000,19651555.8,7558702.109173,27210257.909173,382499.619179
2643,1,г. Биробиджан,99701000,19651555.8,7558702.109173,27210257.909173,382499.619179


In [411]:
income_transferts_2020 = income_transferts_2020_raw.copy()

##### Rename columns and drop useless

In [412]:
income_transferts_2020_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [413]:
income_transferts_2020.columns = income_transferts_2020_columns

In [414]:
# drop useless columns with name '-'
income_transferts_2020 = income_transferts_2020.drop('-', axis=1)

##### Cut rows and drop NaNs

In [415]:
# cut rows
income_transferts_2020 = transform_cut_rows(income_transferts_2020, 4, 0)

In [416]:
# drop rows with NaNs in 'index0' column
income_transferts_2020 = \
    income_transferts_2020[income_transferts_2020['index1'].notna()]

##### Normalize OKTMO

In [417]:
# make OKTMO column values str
income_transferts_2020['ОКТМО'] = income_transferts_2020['ОКТМО'].astype(str)

In [418]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2020['ОКТМО'] = \
    income_transferts_2020['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [419]:
# create index level0 column
income_transferts_2020['index0'] = np.NaN

In [420]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2020 = (income_transferts_2020
                          .assign(index0=income_transferts_2020['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2020['index0'])))

In [421]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2020 = normalize_nenec(income_transferts_2020, 'index0')

In [422]:
# the same correction for 'Тюменская область'
income_transferts_2020 = normalize_khantymanci(income_transferts_2020, 'index0')

In [423]:
# fill '-' values in index0 column
income_transferts_2020 = transform_fill_values_by_previous(
    income_transferts_2020, kind='column', column_name='index0')

##### Level1 for Multiindex

In [424]:
# change region name in 'index1' column to 'Всего'
income_transferts_2020['index1'] = income_transferts_2020.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [425]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2020['index1'] = \
    income_transferts_2020['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [426]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [427]:
# create regions list
regions_list2012 = list(set(income_transferts_2020['index0']))
regions_list2012.sort()

In [428]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2020 = calculate_sum_transferts(
    data=income_transferts_2020,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [429]:
# count regions
regions_count2012 = len(set(income_transferts_2020['index0'].values))
regions_count2012

83

##### Create Multiindex

In [430]:
# columns to multiindex
income_transferts_2020 = transform_make_index_from_columns(
    income_transferts_2020, ['index0', 'index1'])

In [431]:
# drop columns, that became multiindexes
income_transferts_2020 = income_transferts_2020.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [432]:
# round
for column in income_transferts_2020.columns:
    try:
        income_transferts_2020[column] = \
            income_transferts_2020[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2016 Check for Discrepancy</font>

In [433]:
# OKTMO check for NaN
income_transferts_2020[income_transferts_2020['ОКТМО'].isna()].any().any()

False

In [434]:
# all data check for NaN
income_transferts_2020[income_transferts_2020.isna().any(axis=1)].any().any()

True

In [435]:
income_transferts_2020[income_transferts_2020['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [436]:
income_transferts_2020[income_transferts_2020['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [437]:
income_transferts_2020[income_transferts_2020['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [438]:
income_transferts_2020[income_transferts_2020['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [439]:
income_transferts_2020[income_transferts_2020['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,449865535.1,133413326.3,5.832789e+08,675188.3
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,920897253.0,199025707.3,1.119923e+09,1486001.4
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,531177158.0,71653948.5,6.028311e+08,2492005.1


In [440]:
income_transferts_2020[income_transferts_2020['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,296500160.0,134446202.3,430946362.3,757286.1
Ненецкий автономный округ (Архангельская область),Всего,11,50826149.0,11922724.9,62748873.9,1738525.4


In [441]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2016_dupl_list = check_oktmo_duplication(
    data=income_transferts_2020, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2016_dupl_list

['Тверская область', 'Кемеровская область - Кузбасс', 'Ростовская область']

In [442]:
income_transferts_2020.loc[income_transferts_2020['ОКТМО'].isin(municipal_town_dict['Алтайский край'])]

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000   98592778.5   
               Городские округа      01 700 000  268900130.5   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          92751510.9        191344289.4   
               Городские округа             108799078.8        377699209.3   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           186104.3  
               Городские округа               295401.3

##### <font color='#2A5899'>2016 Final Result</font>

In [443]:
income_transferts_2020.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  367492909.0   
               Муниципальные районы  01 600 000   98592778.5   
               Алейский              01 601 000    1591328.9   
               Алтайский             01 602 000    1769736.8   
               Баевский              01 603 000     804125.3   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        201550589.7        569043498.8   
               Муниципальные районы          92751510.9        191344289.4   
               Алейский                       1135974.9          2727303.8   
               Алтайский                      2092842.4          3862579.2   
               Баевский                        916818.9          1720944.2   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          481505.6  
               Муниципальные районы           186104.3  
               Алейский                       201962.7  
               Алтайский                      148298.4  
               Баевский                       197991.7

In [444]:
get_data_two_level(
    data=income_transferts_2020,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,2762633018.7,715024640.4,3477657659.1,1045756.6


In [445]:
income_transferts_2020.loc['Ненецкий автономный округ (Архангельская область)']

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,11,50826149.0,11922724.9,62748873.9,1738525.4
Муниципальные районы,11 800 000,35976672.0,7762359.1,43739031.1,988452.7
Заполярны йрайон,11 811 000,21127195.0,3601993.3,24729188.3,1308007.4
Городские округа,11 850 000,14849477.0,4160365.8,19009842.8,750072.7
г.Нарьян-Мар,11 851 000,14849477.0,4160365.8,19009842.8,750072.7


In [446]:
datasets_list.append(income_transferts_2020)

#### <font color='#AF4035'>2021 год</font>

##### Regions OKTMO codes

In [447]:
path_2021 = \
    ('/Users/a.chasovsky/Library/CloudStorage/GoogleDrive-alexanderchasovsky@gmail.com/'
     'Мой диск/Data Science Projects/pers_economics_rus/data/00-Region-codes/')
os.chdir(path_2021)
%run 2021.ipynb
os.chdir(current_path)

##### Исправления:
- Магаданская область: '44 000 000' -> '44'

##### Data

In [448]:
income_transferts_2021_link = \
    ('data/02-Population/02-Standard-of-living/01-Income/'
     'urov-14a2021.xlsx')

In [449]:
income_transferts_2021_raw = pd.read_excel(income_transferts_2021_link)

In [450]:
income_transferts_2021_raw.head(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"""Объем социальных выплат населению и налогооб...",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Код ОКТМО,Налогооблагаемые денежные доходы физических ли...,"Социальные и другие выплаты, тыс.рублей",Объем социальных выплат населению и налогообла...,Объем социальных выплат и налогооблагаемых ден...
3,NaN,NaN,1,2,3,4,5
4,NaN,Алтайский край,01,NaN,NaN,NaN,NaN
5,NaN,Муниципальные районы,01 600 000,122551388.057801,96835379.023917,219386767.081718,216932.608482
6,1,Алейский,1601000,2147497.52735,1180669.010353,3328166.537703,254252.600283
7,2,Алтайский,1602000,2143931.779109,2192810.318364,4336742.097473,165638.304846
8,3,Баевский,1603000,972777.317738,971883.795473,1944661.113211,230847.710495
9,4,Бийский,1604000,2657963.790435,3006480.308269,5664444.098705,189490.653287


In [451]:
income_transferts_2021_raw.tail(10)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
2659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2660,NaN,Еврейская автономная область,99,NaN,NaN,NaN,NaN
2661,NaN,Муниципальные районы,99600000,16634219,8423691.38444,25057910.38444,293109.257041
2662,1,Биробиджанский,99605000,1333210.666667,1135702.246647,2468912.913314,227633.497447
2663,2,Ленинский,99610000,2335266,1405910.44841,3741176.44841,212036.75178
2664,3,Облученский,99620000,7981012.333333,2684688.415341,10665700.748674,438701.083772
2665,4,Октябрьский,99625000,1503002.666667,903138.655219,2406141.321886,259059.143183
2666,5,Смидовичский,99630000,3481727.333333,2294251.618824,5775978.952157,246836.707357
2667,NaN,Городские округа,99700000,21231514.333333,8357370.60556,29588884.938893,424639.565713
2668,1,г. Биробиджан,99701000,21231514.333333,8357370.60556,29588884.938893,424639.565713


In [452]:
income_transferts_2021 = income_transferts_2021_raw.copy()

##### Rename columns and drop useless

In [453]:
income_transferts_2021_columns = [
    '-', 'index1', 'ОКТМО', 'Доходы', 'Социальные выплаты',
    'Суммарные выплаты', 'Выплаты на 1 чел.'
]

In [454]:
income_transferts_2021.columns = income_transferts_2021_columns

In [455]:
# drop useless columns with name '-'
income_transferts_2021 = income_transferts_2021.drop('-', axis=1)

##### Cut rows and drop NaNs

In [456]:
# cut rows
income_transferts_2021 = transform_cut_rows(income_transferts_2021, 4, 0)

In [457]:
# drop rows with NaNs in 'index0' column
income_transferts_2021 = \
    income_transferts_2021[income_transferts_2021['index1'].notna()]

##### Normalize OKTMO

In [458]:
# make OKTMO column values str
income_transferts_2021['ОКТМО'] = income_transferts_2021['ОКТМО'].astype(str)

In [459]:
# normalize oktmo - remove spaces and other dirt
income_transferts_2021['ОКТМО'] = \
    income_transferts_2021['ОКТМО'].apply(lambda x: normalize_oktmo(x))

##### Level0 for Multiindex

In [460]:
# create index level0 column
income_transferts_2021['index0'] = np.NaN

In [461]:
# fill in by region with OKTMO in OKTMO column
income_transferts_2021 = (income_transferts_2021
                          .assign(index0=income_transferts_2021['ОКТМО']
                                  .map(region_oktmo_norm_dict_sorted)
                                  .fillna(income_transferts_2021['index0'])))

In [462]:
# 1) 'Архангельская область' and 
# 'Ненецкий автономный округ (Архангельская область)' have the same OKTMO == 11
# 2) 'Ненецкий автономный округ (Архангельская область)' index allways greater than index of 
# 'Архангельская область'
# knowing this, we replace duplicate 'Архангельская область' by 'Ненецкий автономный округ (Архангельская область)'
income_transferts_2021 = normalize_nenec(income_transferts_2021, 'index0')

In [463]:
# the same correction for 'Тюменская область'
income_transferts_2021 = normalize_khantymanci(income_transferts_2021, 'index0')

In [464]:
# fill '-' values in index0 column
income_transferts_2021 = transform_fill_values_by_previous(
    income_transferts_2021, kind='column', column_name='index0')

##### Level1 for Multiindex

In [465]:
# change region name in 'index1' column to 'Всего'
income_transferts_2021['index1'] = income_transferts_2021.apply(
    lambda x: transferts_various(x['ОКТМО'], x['index1']), axis=1)

In [466]:
# normalize 'Муниципальные районы' and 'Гордские округа' - remove spaces and other dirt
income_transferts_2021['index1'] = \
    income_transferts_2021['index1'].apply(lambda x: normalize_municipal_town(x))

##### Fill rows with sum ('Всего')

In [467]:
# columns for sum of 'Муниципальные районы' and 'Городские округа'
columns_summary2012 = [
    'Доходы', 'Социальные выплаты', 'Суммарные выплаты', 'Выплаты на 1 чел.']

In [468]:
# create regions list
regions_list2012 = list(set(income_transferts_2021['index0']))
regions_list2012.sort()

In [469]:
# count sum of 'Муниципальные районы' and 'Городские округа'
income_transferts_2021 = calculate_sum_transferts(
    data=income_transferts_2021,
    columns=columns_summary2012,
    regions=regions_list2012,
    mapping_dict=municipal_town_dict)

Region Санкт-Петербург not in "Minucipal-Town Dictionary"


##### Count Regions

In [470]:
# count regions
regions_count2012 = len(set(income_transferts_2021['index0'].values))
regions_count2012

83

##### Create Multiindex

In [471]:
# columns to multiindex
income_transferts_2021 = transform_make_index_from_columns(
    income_transferts_2021, ['index0', 'index1'])

In [472]:
# drop columns, that became multiindexes
income_transferts_2021 = income_transferts_2021.drop(['index0', 'index1'], axis=1)

##### Round (one symbol)

In [473]:
# round
for column in income_transferts_2021.columns:
    try:
        income_transferts_2021[column] = \
            income_transferts_2021[column].astype(float).round(1)
    except ValueError:
        pass

##### <font color='#2A5899'>2016 Check for Discrepancy</font>

In [474]:
# OKTMO check for NaN
income_transferts_2021[income_transferts_2021['ОКТМО'].isna()].any().any()

False

In [475]:
# all data check for NaN
income_transferts_2021[income_transferts_2021.isna().any(axis=1)].any().any()

True

In [476]:
income_transferts_2021[income_transferts_2021['Доходы'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [477]:
income_transferts_2021[income_transferts_2021['Социальные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [478]:
income_transferts_2021[income_transferts_2021['Суммарные выплаты'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [479]:
income_transferts_2021[income_transferts_2021['Выплаты на 1 чел.'] == 0]

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.


In [480]:
income_transferts_2021[income_transferts_2021['ОКТМО'] == '71']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Тюменская область,Всего,71,609145063.1,146729987.8,7.558751e+08,851202.4
Ханты-Мансийский автономный округ - Югра (Тюменская область),Всего,71,979283935.3,209806704.4,1.189091e+09,1559855.8
Ямало-Ненецкий автономный округ (Тюменская область),Всего,71,606352210.1,80043506.1,6.863957e+08,3601131.4


In [481]:
income_transferts_2021[income_transferts_2021['ОКТМО'] == '11']

,,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Архангельская область,Всего,11,362958129.0,145516657.5,508474786.5,1252724.5
Ненецкий автономный округ (Архангельская область),Всего,11,45888319.7,9717011.0,55605330.6,2578201.1


In [482]:
# check if there are duplications in municipal, town and others OKTMO
check_oktmo_2016_dupl_list = check_oktmo_duplication(
    data=income_transferts_2021, 
    column_check='ОКТМО',
    normalize_oktmo_list=municipal_town_dict)

check_oktmo_2016_dupl_list

['Амурская область',
 'Архангельская область',
 'Брянская область',
 'Нижегородская область',
 'Тверская область',
 'Камчатский край',
 'Кемеровская область - Кузбасс',
 'Костромская область',
 'Курганская область',
 'Мурманская область',
 'Новгородская область',
 'Ростовская область',
 'Ямало-Ненецкий автономный округ (Тюменская область)',
 'Забайкальский край']

In [483]:
(income_transferts_2021
 .loc[income_transferts_2021['ОКТМО']
 .isin(municipal_town_dict['Алтайский край'])])

ОКТМО       Доходы  \
Алтайский край Муниципальные районы  01 600 000  122551388.1   
               Городские округа      01 700 000  328209476.9   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Муниципальные районы          96835379.0        219386767.1   
               Городские округа             116515317.9        444724794.8   

                                     Выплаты на 1 чел.  
Алтайский край Муниципальные районы           216932.6  
               Городские округа               349914.4

##### <font color='#2A5899'>2016 Final Result</font>

In [484]:
income_transferts_2021.head()

ОКТМО       Доходы  \
Алтайский край Всего                         01  450760864.9   
               Муниципальные районы  01 600 000  122551388.1   
               Алейский              01 601 000    2147497.5   
               Алтайский             01 602 000    2143931.8   
               Баевский              01 603 000     972777.3   

                                     Социальные выплаты  Суммарные выплаты  \
Алтайский край Всего                        213350696.9        664111561.9   
               Муниципальные районы          96835379.0        219386767.1   
               Алейский                       1180669.0          3328166.5   
               Алтайский                      2192810.3          4336742.1   
               Баевский                        971883.8          1944661.1   

                                     Выплаты на 1 чел.  
Алтайский край Всего                          566847.0  
               Муниципальные районы           216932.6  
               Алейский                       254252.6  
               Алтайский                      165638.3  
               Баевский                       230847.7

In [485]:
get_data_two_level(
    data=income_transferts_2021,
    level0='Московская область',
    level1='Всего',
    kind='index')

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,46,3416106090.7,736227813.5,4152333904.2,542266.8


In [486]:
income_transferts_2021.loc['Брянская область']

,ОКТМО,Доходы,Социальные выплаты,Суммарные выплаты,Выплаты на 1 чел.
Всего,15,246143014.0,122006475.6,368149489.6,885320.4
Муниципальные округа,15 500 000,10609807.4,6827268.8,17437076.2,254055.2
Стародубский,15 501 000,6801975.8,3905711.8,10707687.6,299289.7
Жуковский,15 502 000,3807831.6,2921557.1,6729388.7,204802.1
Муниципальные районы,15 600 000,80339987.1,55000948.7,135340935.8,248796.7
Брасовский,15 604 000,2057558.2,1881066.3,3938624.5,210497.8
Брянский,15 608 000,25097406.0,5215447.3,30312853.3,473890.1
Выгоничский,15 610 000,5446608.1,1627679.2,7074287.3,366315.6
Гордеевский,15 611 000,517655.9,1191994.6,1709650.5,170029.9
Дубровский,15 612 000,1538118.9,1531245.4,3069364.3,187843.6


In [487]:
datasets_list.append(income_transferts_2021)

##### <font color='#AF4035'> Export Dataset to Excel

In [488]:
sheet_names_list = []
for i in range(2010, 2022):
    sheet_names_list.append(str(i))

In [489]:
print(sheet_names_list)

['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']


In [490]:
for dataset, sheet in zip(datasets_list, sheet_names_list):
    saveit_excel(
        data=dataset,
        filename=dir,
        path=save_excel,
        sheet=sheet
    )

'2010' sheet created if file '01020103-SOL-Transferts.xlsx'
'2011' sheet created if file '01020103-SOL-Transferts.xlsx'
'2012' sheet created if file '01020103-SOL-Transferts.xlsx'
'2013' sheet created if file '01020103-SOL-Transferts.xlsx'
'2014' sheet created if file '01020103-SOL-Transferts.xlsx'
'2015' sheet created if file '01020103-SOL-Transferts.xlsx'
'2016' sheet created if file '01020103-SOL-Transferts.xlsx'
'2017' sheet created if file '01020103-SOL-Transferts.xlsx'
'2018' sheet created if file '01020103-SOL-Transferts.xlsx'
'2019' sheet created if file '01020103-SOL-Transferts.xlsx'
'2020' sheet created if file '01020103-SOL-Transferts.xlsx'
'2021' sheet created if file '01020103-SOL-Transferts.xlsx'


### Save Dataset

In [491]:
for dataset, name in zip(datasets_list, np.arange(371, 371+12)):
    name_transformed = str.join('.', list(str(name)))
    economics_data[name_transformed] = dataset
    print(f'Dataset {name_transformed} added')

Dataset 3.7.1 added
Dataset 3.7.2 added
Dataset 3.7.3 added
Dataset 3.7.4 added
Dataset 3.7.5 added
Dataset 3.7.6 added
Dataset 3.7.7 added
Dataset 3.7.8 added
Dataset 3.7.9 added
Dataset 3.8.0 added
Dataset 3.8.1 added
Dataset 3.8.2 added


In [492]:
saveit(
    file=economics_data,
    name='economics_data',
    dir=save_df
)

File 'economics_data' saved in directory 'data/00-Datasets/'


### Save Session

In [493]:
save_session(dir)

### Execution time

In [494]:
print(f'Execution time: {stopwatch.stop(start)}')

Execution time: 0:07:46
